In [ ]:
#all_data.set_index(['PassengerId'], inplace=True)

all_data.loc[:, 'LastName'] = all_data['Name'].apply(lambda x: str.split(x, ',')[0])
    
fare_df = all_data.loc[all_data['FamilySize'] > 1, ['LastName', 'Fare', 'FamilySize']].iloc[:891]
print('Number of families in train: {}'.format(fare_df.shape[0]))
fare_diff = (((fare_df.groupby(['LastName', 'FamilySize']).max() 
               - fare_df.groupby(['LastName', 'FamilySize']).min()) != 0).sum() / 891 * 100)
print('Percentage of families with different fares is: {:.1f}%'.format(fare_diff.values[0]))

train_temp = all_data.iloc[:891]
family_df_grpby = train_temp[train_temp['FamilySize'] > 1][['LastName', 'Fare',
                                                            'FamilySize', 'Survived']].groupby(['LastName',
                                                                                                'Fare'])
# Number of rows in each group in train data
family_df = pd.DataFrame(data=family_df_grpby.size(), columns=['Size in train'])

# Survived in each row of each group in train
family_df['Survived total'] = family_df_grpby['Survived'].sum().astype(int)

# Family size in each row of each group in train
family_df['FamilySize'] = family_df_grpby['FamilySize'].mean().astype(int)

print('Whole family survived {:.1f}%'.format(len(family_df[family_df['Size in train'] == 
                                                       family_df['Survived total']]) / len(family_df) * 100))

print('Whole family died {:.1f}%'.format(len(family_df[family_df['Survived total'] == 0])/len(family_df) * 100))

all_data['FamilySurvival'] = 0.5

for grp, grp_df in all_data[['Survived', 'Name', 'LastName', 'Fare', 
                             'SibSp', 'Parch', 'Age', 'Cabin']].groupby(['LastName', 'Fare']):
    for pass_id, row in grp_df.iterrows():
        smax = grp_df.drop(pass_id)['Survived'].max()
        smin = grp_df.drop(pass_id)['Survived'].min()
        
        # If any family member survived
        if (smax == 1.0):
            all_data.loc[pass_id, 'FamilySurvival'] = 1
        # If all family members died
        elif (smin == 0):
            all_data.loc[pass_id, 'FamilySurvival'] = 0
            
print('Number of passenger with family survival information: {}'.format(all_data[all_data['FamilySurvival'] != 0.5].shape[0]))

In [ ]:
train_temp = all_data.iloc[:891]
ticket_grpby = train_temp.groupby('Ticket')

# Number of rows(passengers) in each group by ticket
ticket_df = pd.DataFrame(data=ticket_grpby.size(), columns=['Size in train'])

# Survived in each row of each group in train
ticket_df['Survived total'] = ticket_grpby['Survived'].sum().astype(int)

# Passengers from different family that shared ticket
ticket_df['Not family'] = ticket_grpby['LastName'].unique().apply(len)

ticket_df = ticket_df[(ticket_df['Size in train'] > 1) & (ticket_df['Not family']>1)]

print('Number of groups in training set that is not family: {}'.format(len(ticket_df)))

print('Whole group died: {:.2f}%'.format(100/len(ticket_df) * len(ticket_df[ticket_df['Survived total'] == 0])))
print('Whole group survived: {:.2f}%'.format(100/len(ticket_df) * len(ticket_df[ticket_df['Survived total'] == ticket_df['Size in train']])))

for _, grp_df in all_data.groupby('Ticket'):
    # grp_df - passengers which share the same ticket
    if len(grp_df) != 1:
        for ind, row in grp_df.iterrows():
            if (row['FamilySurvival'] == 0) | (row['FamilySurvival'] == 0.5):
                smax = grp_df.drop(ind)['Survived'].max()
                smin = grp_df.drop(ind)['Survived'].min()
                passID = row['PassengerId']
                if smax == 1:
                    all_data.loc[all_data['PassengerId'] == passID, 'FamilySurvival'] = 1
                elif smin == 0:
                    all_data.loc[all_data['PassengerId'] == passID, 'FamilySurvival'] = 0
                    
print('Number of passenger with family/group survival information: ',
      all_data[all_data['FamilySurvival'] != 0.5].shape[0])